<a href="https://colab.research.google.com/github/terrence-ou/DL_Playground/blob/main/%5BFT%5DViT_Baseline_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Tue Mar 28 17:18:50 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    48W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# with open("/root/.kaggle/kaggle.json", "w+") as f:
#     f.write('{"username":"tingsongou","key":"f6cc12e409af2aa8bdf85bca88a3d434"}') # Put your kaggle username & key here

# !chmod 600 /root/.kaggle/kaggle.json

# 1. Start

https://github.com/poloclub/diffusiondb \\
https://www.kaggle.com/code/shoheiazuma/diffusiondb-data-cleansing \\
https://www.kaggle.com/code/shoheiazuma/stable-diffusion-vit-baseline-train \\
https://www.kaggle.com/code/shoheiazuma/stable-diffusion-vit-baseline-inference

In [ ]:
!pip install -q sentence-transformers datasets tqdm timm alive-progress

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 KB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.0/74.0 KB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 104.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 KB 26.6 MB/s eta 0

In [ ]:
import numpy as np
from scipy import spatial

from PIL import Image
import datasets
from tqdm.notebook import tqdm
import timm
from timm.utils import AverageMeter

import os
import unicodedata
import pandas as pd
from sentence_transformers import SentenceTransformer

import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torch import autocast

from torchvision import transforms
from urllib.request import urlretrieve
import pandas as pd

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

In [ ]:
table_url = f'https://huggingface.co/datasets/poloclub/diffusiondb/resolve/main/metadata.parquet'
urlretrieve(table_url, 'metadata.parquet')

('metadata.parquet', <http.client.HTTPMessage at 0x7fba7ad89850>)

In [ ]:
def is_english_only(string):
    for s in string:
        cat = unicodedata.category(s)         
        if not cat in ['Ll', 'Lu', 'Nd', 'Po', 'Pd', 'Zs']:
            return False
    return True

In [ ]:
!mkdir imgs
!cd imgs
!python download.py -r 100 -z

In [ ]:
len(os.listdir("/content/"))

99107

In [ ]:
df = pd.read_parquet('/content/metadata.parquet', columns=['image_name', 'prompt', 'width', 'height'])
df = df[(df['width'] == 512) & (df['height'] == 512)]
df['prompt'] = df['prompt'].str.strip()
df = df[df['prompt'].map(lambda x: len(x.split())) >= 5]
df = df[~df['prompt'].str.contains('^(?:\s*|NULL|null|NaN)$', na=True)]
df = df[df['prompt'].apply(is_english_only)]
df['head'] = df['prompt'].str[:15]
df['tail'] = df['prompt'].str[-15:]
df.drop_duplicates(subset='head', inplace=True)
df.drop_duplicates(subset='tail', inplace=True)
df.reset_index(drop=True, inplace=True)

# for i in tqdm(range(1, 2000, 100)):
#     image_dir = f'/kaggle/input/diffusiondb-2m-part-{str(i).zfill(4)}-to-{str(i+99).zfill(4)}-of-2000/'
#     images = os.listdir(image_dir)
#     df.loc[df['image_name'].isin(images), 'filepath'] = image_dir + df['image_name']

images = os.listdir("/content/")
df.loc[df['image_name'].isin(images), 'filepath'] = "/content/" + df['image_name']

df = df[['filepath', 'prompt']].copy()
# assert not df['filepath'].isnull().any()
df = df.dropna(subset=['filepath'])
df.to_csv('diffusiondb.csv', index=False)

In [ ]:
class Config():
    model_name = "vit_base_patch16_224"
    input_size = 224
    batch_size = 256
    num_epochs = 10
    lr = 1e-4

In [ ]:
# %%capture
# data = datasets.load_dataset('poloclub/diffusiondb', 'large_random_50k', split="train") # 5k data takes about 3.3G

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# num_data = len(data)
# display(data[3]["image"])
# print(data[3]["prompt"])

In [ ]:
class DiffusionDataset(Dataset):
    def __init__(self, df, transform):
        self.df = df
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(row['filepath'])
        image = self.transform(image)
        prompt = row['prompt']
        return image, prompt


class DiffusionCollator:
    def __init__(self):
        self.st_model = SentenceTransformer(
            'sentence-transformers/all-MiniLM-L6-v2',
            device='cpu'
        )
    
    def __call__(self, batch):
        images, prompts = zip(*batch)
        images = torch.stack(images)
        prompt_embeddings = self.st_model.encode(
            prompts, 
            show_progress_bar=False, 
            convert_to_tensor=True
        )
        return images, prompt_embeddings

In [ ]:
def get_dataloaders(train_split, valid_split, input_size, batch_size):

    transform = transforms.Compose([
        transforms.Resize(input_size),
        transforms.ToTensor(),
        transforms.CenterCrop(input_size),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

    train_dataset = DiffusionDataset(train_split, transform)
    valid_dataset = DiffusionDataset(valid_split, transform)
    collate_fn = DiffusionCollator()

    train_loader = DataLoader(train_dataset, 
                              batch_size=batch_size,
                              shuffle=True,
                              pin_memory=True,
                              collate_fn=collate_fn)
    valid_loader = DataLoader(valid_dataset,
                              batch_size=batch_size,
                              shuffle=False,
                              pin_memory=True,
                              collate_fn=collate_fn)
    
    return train_loader, valid_loader
     

In [ ]:
# train_split, valid_split = random_split(data, [int(num_data * 0.7), int(num_data * 0.3)])
# train_loader, valid_loader = get_dataloaders(train_split, valid_split, 224, 32)

In [ ]:
def cosine_similarity(y_trues, y_preds):
    return np.mean([
        1 - spatial.distance.cosine(y_true, y_pred)
        for y_true, y_pred in zip(y_trues, y_preds)
    ])

In [ ]:
def train(train_split, valid_split, model_name, input_size, batch_size, num_epochs, lr):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Training device: {device}")

    train_loader, valid_loader = get_dataloaders(train_split, valid_split, input_size, batch_size)
    model = timm.create_model(
        model_name,
        pretrained=True,
        num_classes=384
    )

    model.set_grad_checkpointing()
    model.to(device)

    total_iters = num_epochs * len(train_loader)

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, amsgrad=True)
    criterion = torch.nn.CosineEmbeddingLoss()

    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=total_iters, eta_min=1e-6)
    scaler = torch.cuda.amp.GradScaler()

    best_score = -1.0


    for epoch in range(num_epochs):
        torch.cuda.empty_cache()

        train_meters = {
            "loss": AverageMeter(),
            "cos": AverageMeter()
        }

        model.train()

        train_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

        for X, y in train_loader:
            X, y = X.to(device), y.to(device)
            
            optimizer.zero_grad()
            X_out = model(X)
            target = torch.ones(X.shape[0]).to(device)
            loss = criterion(X_out, y, target)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            scheduler.step()

            train_loss = loss.item()
            train_cos = cosine_similarity(
                X_out.detach().cpu().numpy(),
                y.detach().cpu().numpy()
            )

            train_meters["loss"].update(train_loss, n=X.shape[0])
            train_meters["cos"].update(train_cos, n=X.shape[0])

            train_bar.set_postfix(
                loss="{:.04f}".format(train_meters["loss"].avg),
                cos="{:.04f}".format(train_meters["cos"].avg),
                lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
            train_bar.update()
        train_bar.close()

        print("Epoch {:d}\n\t train loss={:.4f}, train cos={:.4f}".format(
            epoch + 1,
            train_meters["loss"].avg,
            train_meters["cos"].avg))
        
        valid_meters = {
            "loss": AverageMeter(),
            "cos": AverageMeter()
        }

        valid_bar = tqdm(total=len(valid_loader), dynamic_ncols=True, leave=False, position=0, desc='valid')

        model.eval()
        for X, y in valid_loader:
            X, y = X.to(device), y.to(device)

            with torch.no_grad():
                X_out = model(X)
                target = torch.ones(X_out.shape[0]).to(device)
                loss = criterion(X_out, y, target)

                val_loss = loss.item()
                val_cos = cosine_similarity(
                    X_out.detach().cpu().numpy(),
                    y.detach().cpu().numpy()
                )
            
            valid_meters["loss"].update(val_loss, n=X.shape[0])
            valid_meters["cos"].update(val_cos, n=X.shape[0])

            valid_bar.set_postfix(
                loss="{:.04f}".format(valid_meters["loss"].avg),
                cos="{:.04f}".format(valid_meters["cos"].avg),
                lr="{:.07f}".format(float(optimizer.param_groups[0]['lr'])))
            valid_bar.update()
        valid_bar.close()


        print("\t val loss={:.4f}, val cos={:.4f}".format(
            valid_meters["loss"].avg,
            valid_meters["cos"].avg))

        if valid_meters["cos"].avg > best_score:
            best_score = valid_meters["cos"].avg
            print("Saving model...")
            torch.save(model.state_dict(), f"/content/drive/MyDrive/saved_models/{model_name}.pth")
        
        del X, y

In [ ]:
model = timm.create_model(
    Config.model_name,
    pretrained=True,
    num_classes=384
)

In [ ]:
df = pd.read_csv("diffusiondb.csv")

In [ ]:
num_data = len(df)
train_split, valid_split = train_test_split(df, test_size=0.1)
train(train_split, valid_split,
      Config.model_name,
      Config.input_size,
      Config.batch_size,
      Config.num_epochs,
      Config.lr)
     

Training device: cuda


Train:   0%|          | 0/49 [00:00<?, ?it/s]

Epoch 1
	 train loss=0.6142, train cos=0.3858


valid:   0%|          | 0/6 [00:00<?, ?it/s]

	 val loss=0.5745, val cos=0.4255
Saving model...


...

	 val loss=0.5310, val cos=0.4690


Train:   0%|          | 0/49 [00:00<?, ?it/s]

Epoch 10
	 train loss=0.2837, train cos=0.7163


valid:   0%|          | 0/6 [00:00<?, ?it/s]

	 val loss=0.5338, val cos=0.4662
